In [1]:
using OffsetArrays, Parameters, Plots

In [2]:
gr(fmt = :png)

Plots.GRBackend()

In [3]:
N = 3 # number of firms per cohort
cohorts = (N, N, N) # cohorts
K = length(cohorts) # number of cohorts
f = fill(1/(N+1)^K, (N+1)^K); # awareness set distribution

In [4]:
q = fill(1.0, K) # K-vector of qualities
params = (cohorts = cohorts, Γ_bar = 1.0, Ω = 1.0, q = q, σ = 0.5, κ = 1.5)

(cohorts = (3, 3, 3), Γ_bar = 1.0, Ω = 1.0, q = [1.0, 1.0, 1.0], σ = 0.5, κ = 1.5)

In [5]:
# Demand function with multiple cohorts
# p_i: the price of product of firm i (can be different from p[b])
# b: the cohort firm i belongs to
# p: K-vector of prices across cohorts
# f: awareness set distribution
# params: model parameters
function demand(p_i, b, p, f, params)
    @unpack cohorts, Γ_bar, Ω, q, σ, κ = params
    N = cohorts[1]
    K = length(cohorts)
    
    f = reshape(f, Tuple(fill(0:N, K)))
    demand_value = 0
    summand(n) = sum(Tuple(n) .* (p ./ q).^(-1/σ)) 
    for n in CartesianIndices(f)
        if (n[b] > 0)
            demand_value += n[b] * f[n] * (summand(n) + p_i^(-1/σ) - p[b]^(-1/σ))^(σ*(κ-1)-1)
        end
    end
    demand_value *= Γ_bar^(1-κ)*Ω*(q[b]^(1/σ))*(p_i^(-1/σ-1))
    return demand_value
end

demand (generic function with 1 method)

In [6]:
b = 1 # the cohort firm i belongs to
p = fill(1.0, K); # K-vector of prices

In [ ]:
# plot the demand function
plot(range(0.1,stop=3.0,length=100), p_i -> demand(p_i, b, p, f, params),
    label = "demand(p_i)", lw = 3)

In [ ]:
# confirm that the demand function is decreasing in price
@assert all(diff((p_i -> demand(p_i, b, p, f, params)).(1:100)) .< 0)